# Overview

In February 19, 2019, Databricks [announced](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html) the release of the time travel feature. 

> With this new feature, Delta automatically versions the big data that you store in your data lake, and you can access any historical version of that data. This temporal data management simplifies your data pipeline by making it easy to audit, roll back data in case of accidental bad writes or deletes, and reproduce experiments and reports. Your organization can finally standardize on a clean, centralized,  versioned big data repository in your own cloud storage for your analytics.

In this notebook we will explore this feature as well as the problems it is looking to solve.

## Reminders
Recall that delta lake is built on top of Apache Spark. Delta lake is the storage layer while Spark is the execution layer.

As such Spark is providing a significant portion of the tech stack one would interact with when using delta lake. We load our data using spark, we transform our data using spark, we are using spark. As such, we must be aware of some of the tools in the spark tool belt that will be relevant to the aspect of time travel. 

The Spark provides a python api called PySpark. PySpark allows python to communicate with the lower level APIs of the execution layer; specifically the Dataframe API which loads and manipulates dataframes, and the SQL API which allows us to execute SQL queries against dataframes the way we would against traditional sql tables (sits on top of dataframes).

At this point, to exploit ALL the time travel functionality, there are some things we must do with the spark SQL API. We will point those out as they are encountered.

# 1. Challenges Being Addressed

## 1.1. Audit data changes
Auditing data changes is critical from both in terms of data compliance as well as simple debugging to understand how data has changed over time. Traditionally the audit was tightly coupled with the pipelines responsible for delivering the data. This feature standardizes the audit regardless of how the data got into the data lake.

## 1.2. Reproduce experiments & reports
During model training, data scientists run various experiments with different parameters on a given set of data. When scientists revisit their experiments after a period of time to reproduce the models, typically the source data has been modified by upstream pipelines. Lot of times, they are caught unaware by such upstream data changes and hence struggle to reproduce their experiments. Some scientists and organizations engineer best practices by creating multiple copies of the data, leading to increased storage costs. The same is true for analysts generating reports.

## 1.3. Rollbacks
Data pipelines can sometimes write bad data for downstream consumers. This can happen because of issues ranging from infrastructure instabilities to messy data to bugs in the pipeline. For pipelines that do simple appends to directories or a table, rollbacks can easily be addressed by date-based partitioning. With updates and deletes, this can become very complicated, and data engineers typically have to engineer a complex pipeline to deal with such scenarios. This is no longer the case with time travel.

# 2. Under The Hood: How does time travel work?

As you write into a Delta table or directory, every operation is automatically versioned. And deltalake provides you with an API for accessing data as of a specified timestamp. But how does this work under the hood?

## 2.1. A Brief Explanation of what delta lake is and how it works
A great video can be found [here](https://www.youtube.com/watch?v=o-zcZvfSUyIhttps://www.youtube.com/watch?v=o-zcZvfSUyI) and a complimentary article can be found [here](https://databricks.com/blog/2019/08/21/diving-into-delta-lake-unpacking-the-transaction-log.html).

A Delta lake is a collection of directories and delta tables stored in a data lake. As the name suggests, a delta table is a data table composed of deltas; in other words, a delta table is the result of applying a series of transactions to an initial state. We will see that a delta table is not a file, but a carefully managed directory consisting of files representing the transactions and data which compose the delta table. When we load a table into memory we are actually assembling the table from data files stored in the directory.

As such, when we perform operations on the delta table (i.e. store a snapshot of the state of a spark dataframe), we see the files in this dirctory are modified. 

The delta table directory contains several types of files
1. Compressed parquet files
2. Checksums to validate the parwuet files
3. Transaction logs
4. Checkpoint files (cached state to improve computational efficiency)

The delta table has a log directory which stores json log files which record operations beind performed on the delta table. These operations describe how we transform the data from one state to another state. The operations can be categorized as follows:
- update metadata
- add parquet file
- remove parquet file
- set transaction (record idempotent transaction id)
- change transaction protocol version

This log directory is extremely important because it is what provides the information for calculating versions and of data and their coresponding timestamps. The log directory consists of transaction files, each transaction file pertains to a specific version of data. File '00000000000000000000.json' coresponds to version 0 while file '00000000000000000000.json' coresponds to version 1 of the delta table. Each transaction file has a modified data assigned by the operating system. This timestamp attaches a datatime to our version.

Below we can see an example detlat table which i have stored in the `data/time-travel-demo.delta` directory

```
[root@pc]# tree data/time-travel-demo.delta
data/time-travel-demo.delta/
|-- _delta_log
|   |-- 00000000000000000000.json
|   `-- 00000000000000000001.json
|-- part-00000-41e1a8cd-15d5-4263-9413-f649ad1d51da-c000.snappy.parquet
`-- part-00000-e8891ae6-e9d4-449a-8531-de44d41f7669-c000.snappy.parquetdata
```

We see that the directory is filled with files (the crc files are hidden)

```
[root@pc]# ls -la data/time-travel-demo.delta/

total 3
drwxr-xr-x 1 root root   5 May 20 19:29 .
drwxr-xr-x 1 root root   2 May 20 18:42 ..
-rw-r--r-- 1 root root  16 May 20 18:42 .part-00000-41e1a8cd-15d5-4263-9413-f649ad1d51da-c000.snappy.parquet.crc
-rw-r--r-- 1 root root  16 May 20 19:29 .part-00000-e8891ae6-e9d4-449a-8531-de44d41f7669-c000.snappy.parquet.crc
drwxr-xr-x 1 root root   2 May 20 19:29 _delta_log
-rw-r--r-- 1 root root 735 May 20 18:42 part-00000-41e1a8cd-15d5-4263-9413-f649ad1d51da-c000.snappy.parquet
-rw-r--r-- 1 root root 946 May 20 19:29 part-00000-e8891ae6-e9d4-449a-8531-de44d41f7669-c000.snappy.parquet
```

And we can get the timestamp for the modification of the file

```
[root@pc]# stat data/time-travel-demo.delta/_delta_log/*

  File: 'data/time-travel-demo.delta/_delta_log/00000000000000000000.json'
  Size: 819       	Blocks: 2          IO Block: 262144 regular file
Device: 44h/68d	Inode: 1099511789237  Links: 1
Access: (0644/-rw-r--r--)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2022-05-20 18:42:13.100546536 +0000
Modify: 2022-05-20 18:42:13.341529695 +0000
Change: 2022-05-20 18:42:13.691505237 +0000
 Birth: -
  File: 'data/time-travel-demo.delta/_delta_log/00000000000000000001.json'
  Size: 1049      	Blocks: 3          IO Block: 262144 regular file
Device: 44h/68d	Inode: 1099511789264  Links: 1
Access: (0644/-rw-r--r--)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2022-05-20 19:29:50.672776623 +0000
Modify: 2022-05-20 19:29:51.389726484 +0000
Change: 2022-05-20 19:29:51.526716905 +0000
 Birth: -  
 ```
 
**Note**: While delta lake does use this modification timestamp to attach a timestamp to a version number, it is not exact. We will see that timestamps for version numbers only takethe first three digits of the microsecond field. In this example we would see a timestamp of '2022-05-20 19:29:51.389' attached to version 1. 

**Note**: If the filesystem timestamps get messed up or out of order, delta lake will add one milisecond to the previoustimestamp to compute a new timestamp for a version. So the filesystem timestamps may not always line up.

We will see that deltalake provides a utility for calculating the timestamps ov each version so we dont have to worry!

### 2.1.1. A note on concurrency

In certain situations it is possible that multiple users will be reading and writing data to the same delta table at the same time. The implimentation is described in [this article](https://databricks.com/blog/2019/08/21/diving-into-delta-lake-unpacking-the-transaction-log.html) but it is outside the scope of this article.

## 2.1. Fire up spark
We assume you already have a working spark implimentation and you have already installed the delta pip package.

In [1]:
import pyspark
import delta
sparkConf = pyspark.SparkConf()
sparkConf.setAppName("delta-time-travel-demo")
sparkConf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
sparkConf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
sparkConf.set("spark.databricks.delta.stateReconstructionValidation.enabled", "false")
sparkSessionBuilder = pyspark.sql.SparkSession.builder.config(conf=sparkConf)
sparkSession = delta.configure_spark_with_delta_pip(sparkSessionBuilder).getOrCreate()

:: loading settings :: url = jar:file:/usr/lib/spark-3.1.1-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0ab530cd-2901-4322-968e-19fb3fd2fb0d;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.1 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 972ms :: artifacts dl 15ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.1 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runtime;3.5.2 fro

## 2.2. Write data and observe changes

In [2]:
import pandas
pandas_df = pandas.DataFrame({
    "A": [1,2,3,4,5],
    "B": [6,7,8,9,10],
})
spark_df = sparkSession.createDataFrame(pandas_df)
spark_df.show()

+---+---+
|  A|  B|
+---+---+
|  1|  6|
|  2|  7|
|  3|  8|
|  4|  9|
|  5| 10|
+---+---+



 Before writing our data we will determine where we are writing and will cleanup.

In [10]:
import os
import pyprojroot
project_root_dir = pyprojroot.here()
data_directory = os.path.join(project_root_dir, r"Example Data Sets", "deltalake")
table_name = "time-travel-demo.delta"
full_delta_table_path = os.path.join(data_directory, table_name)
escaped_full_delta_table_path = full_delta_table_path.replace(" ", "\ ")

In [11]:
! rm -rf {escaped_full_delta_table_path}

In [12]:
! mkdir -p {escaped_full_delta_table_path}

In [13]:
! ls -la {escaped_full_delta_table_path}

total 0
drwxr-xr-x 1 root root 0 May 25 15:10 .
drwxr-xr-x 1 root root 1 May 25 15:10 ..


We now write the data as a delta table

In [14]:
spark_df.write.format("delta").save(full_delta_table_path)

We again observe our directory

In [15]:
! tree {escaped_full_delta_table_path}

/workspaces/crypto-data/jupyter-pod/ml-training-jupyter-notebooks/Example\ Data\ Sets/deltalake/time-travel-demo.delta
|-- _delta_log
|   `-- 00000000000000000000.json
`-- part-00000-43a222b4-7935-47a8-9163-2e8db05e786f-c000.snappy.parquet

1 directory, 2 files


In [16]:
import json
import pyprojroot
import os

def load_delta_log(log_file_path):
    with open(log_file_path, "r") as file:
        lines = file.readlines()
        proper_json = "[" + ",".join(lines) + "]"
        d = json.loads(proper_json)
        return d

In [17]:
log_file_path = os.path.join(full_delta_table_path, "_delta_log","00000000000000000000.json") 
print(json.dumps(load_delta_log(log_file_path), indent=4))

[
    {
        "commitInfo": {
            "timestamp": 1653491448089,
            "operation": "WRITE",
            "operationParameters": {
                "mode": "ErrorIfExists",
                "partitionBy": "[]"
            },
            "isBlindAppend": true,
            "operationMetrics": {
                "numFiles": "1",
                "numOutputBytes": "735",
                "numOutputRows": "5"
            }
        }
    },
    {
        "protocol": {
            "minReaderVersion": 1,
            "minWriterVersion": 2
        }
    },
    {
        "metaData": {
            "id": "788223d0-0633-4a55-a0b2-52d051ba76ed",
            "format": {
                "provider": "parquet",
                "options": {}
            },
            "schemaString": "{\"type\":\"struct\",\"fields\":[{\"name\":\"A\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"B\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}",
            "partitionColumns": [],
      

It's important to understand the data written in this file. We have the following list of dictionaries. 

1. Information about the commit
2. Information about the transaction protocol
3. [Table metadata](https://docs.databricks.com/delta/delta-batch.html#table-properties) (We will discuss these in more detail later)
4. Information about Add operation


These are effectively our shapshots and we can see a section for add which tells us that a new part was added to the table.

We now modify our data and again write our data to the delta lake

In [18]:
from pyspark.sql.functions import monotonically_increasing_id, row_number, udf
from pyspark.sql.types import IntegerType

C = [11,12,13,14,15]
spark_udf = udf(lambda i: C[i -1], IntegerType())
new_spark_df = spark_df.withColumn("C", spark_udf('A'))
new_spark_df.show()

+---+---+---+
|  A|  B|  C|
+---+---+---+
|  1|  6| 11|
|  2|  7| 12|
|  3|  8| 13|
|  4|  9| 14|
|  5| 10| 15|
+---+---+---+



In [19]:
! ls -la {escaped_full_delta_table_path}

total 2
drwxr-xr-x 1 root root   3 May 25 15:10 .
drwxr-xr-x 1 root root   1 May 25 15:10 ..
-rw-r--r-- 1 root root  16 May 25 15:10 .part-00000-43a222b4-7935-47a8-9163-2e8db05e786f-c000.snappy.parquet.crc
drwxr-xr-x 1 root root   1 May 25 15:10 _delta_log
-rw-r--r-- 1 root root 735 May 25 15:10 part-00000-43a222b4-7935-47a8-9163-2e8db05e786f-c000.snappy.parquet


In [20]:
new_spark_df\
  .write\
  .format("delta")\
  .mode("overwrite")\
  .option("overwriteSchema", "true")\
  .save(full_delta_table_path)

In [21]:
! tree {escaped_full_delta_table_path}

/workspaces/crypto-data/jupyter-pod/ml-training-jupyter-notebooks/Example\ Data\ Sets/deltalake/time-travel-demo.delta
|-- _delta_log
|   |-- 00000000000000000000.json
|   `-- 00000000000000000001.json
|-- part-00000-43a222b4-7935-47a8-9163-2e8db05e786f-c000.snappy.parquet
`-- part-00000-bc0be56e-f617-406e-8993-7106c4b0801f-c000.snappy.parquet

1 directory, 4 files


We can see a new part has now shown up in the log directory. If we look at the log, we can see more entries than just an add (we see an add and a remove operation).

In [22]:
log_file_path = os.path.join(full_delta_table_path, "_delta_log","00000000000000000001.json") 
log = load_delta_log(log_file_path)
print(json.dumps(log, indent=4))

[
    {
        "commitInfo": {
            "timestamp": 1653491527742,
            "operation": "WRITE",
            "operationParameters": {
                "mode": "Overwrite",
                "partitionBy": "[]"
            },
            "readVersion": 0,
            "isBlindAppend": false,
            "operationMetrics": {
                "numFiles": "1",
                "numOutputBytes": "946",
                "numOutputRows": "5"
            }
        }
    },
    {
        "metaData": {
            "id": "788223d0-0633-4a55-a0b2-52d051ba76ed",
            "format": {
                "provider": "parquet",
                "options": {}
            },
            "schemaString": "{\"type\":\"struct\",\"fields\":[{\"name\":\"A\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"B\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"C\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}}]}",
            "partitionColumns": [],
            "configur

Looking closely we can see that a part was removed from the table (the part we added in the first transaction file) and a new part was added.

We can also see timestamps associated with each operation (modificationTime for add and deletionTimestamp for remove operations) as well as a timestamp for the entire atomic operation (commitInfo) and a timestamp for the metadata. We 

## 2.3. Load timestamps for operations from log

Timestamps are somewhat difficult to interpret. We will load the timestamps into datetime objects so they are more human friendly and so we can see what is going on.

In [24]:
import datetime

log0 = load_delta_log(os.path.join(full_delta_table_path, "_delta_log","00000000000000000000.json"))

log0_commit_info = log0[0]["commitInfo"]
log0_commit_info_timestamp = log0_commit_info["timestamp"]
log0_commit_info_dt = datetime.datetime.fromtimestamp(log0_commit_info_timestamp/ 1e3)

log0_metadata_info = log0[2]["metaData"]
log0_metadata_info_timestamp = log0_metadata_info["createdTime"]
log0_metadata_info_dt = datetime.datetime.fromtimestamp(log0_metadata_info_timestamp/ 1e3)

log0_add_info = log0[3]["add"]
log0_add_info_timestamp = log0_add_info["modificationTime"]
log0_add_info_dt = datetime.datetime.fromtimestamp(log0_add_info_timestamp/ 1e3)

log1 = load_delta_log(os.path.join(full_delta_table_path, "_delta_log","00000000000000000001.json"))

log1_commit_info = log1[0]["commitInfo"]
log1_commit_info_timestamp = log1_commit_info["timestamp"]
log1_commit_info_dt = datetime.datetime.fromtimestamp(log1_commit_info_timestamp/ 1e3)

log1_metadata_info = log1[1]["metaData"]
log1_metadata_info_timestamp = log1_metadata_info["createdTime"]
log1_metadata_info_dt = datetime.datetime.fromtimestamp(log0_metadata_info_timestamp/ 1e3)

log1_add_info = log1[2]["add"]
log1_add_info_timestamp = log1_add_info["modificationTime"]
log1_add_info_dt = datetime.datetime.fromtimestamp(log1_add_info_timestamp / 1e3)

log1_remove_info = log1[3]["remove"]
log1_remove_info_timestamp = log1_remove_info["deletionTimestamp"]
log1_remove_info_dt = datetime.datetime.fromtimestamp(log1_remove_info_timestamp/ 1e3)

print("v0 Info:")
print(f"Commit: {log0_commit_info_dt}")
print(f"Add: {log0_add_info_dt}")
print(f"Creation: {log0_metadata_info_dt}")
print("")
print("v1 Info:")
print(f"Commit: {log1_commit_info_dt}")
print(f"Add: {log1_add_info_dt}")
print(f"Remove: {log1_remove_info_dt}")
print(f"Creation: {log1_metadata_info_dt}")

v0 Info:
Commit: 2022-05-25 15:10:48.089000
Add: 2022-05-25 15:10:47.879000
Creation: 2022-05-25 15:10:46.519000

v1 Info:
Commit: 2022-05-25 15:12:07.742000
Add: 2022-05-25 15:12:00.845000
Remove: 2022-05-25 15:12:07.741000
Creation: 2022-05-25 15:10:46.519000


We can see that the timestamp from the commitInfo section occurs the latest which logically makes sense.

# 3. Delta Table History

As mentioned previously, a Delta Table provides a mechanism for tracking changes that occur since its creation. There are a few ways to get this versioning info, we will look at those now.

## 3.1. Viewing table history using the DeltaTable object

Delta lake provides the DeltaTable object which hosts a number of useful functions. One of these is the history function. This function returns a dataframe with information related to the commits made to the delta table and to associated version numbers and timestamps.

In [25]:
from delta.tables import DeltaTable

# Create a pointer tothe delta table (dont confuse with spark dataframe)
deltaTable = DeltaTable.forPath(sparkSession, full_delta_table_path)

# Get the history as a spark dataframe
dt_history_df = deltaTable.history()

# Convert the info to pandas to make the presentation nicer
dt_history_pd_df = dt_history_df.toPandas()
dt_history_pd_df

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,1,2022-05-25 15:12:07.985,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,0.0,None,False,"{'numOutputRows': '5', 'numOutputBytes': '946'...",None
1,0,2022-05-25 15:10:48.657,None,None,WRITE,"{'mode': 'ErrorIfExists', 'partitionBy': '[]'}",None,None,None,NaN,None,True,"{'numOutputRows': '5', 'numOutputBytes': '735'...",None


**Note**: We see some information in some columns is missing. That will be discussed separaetely.**Note**: We see some information in some columns is missing. That will be discussed separaetely.

We see that we can use the version number to get timestamps associated with specific version or look up which version occurred relative to a specific time.

In [26]:
v1_dt = dt_history_pd_df[dt_history_pd_df["version"] == 1]["timestamp"].iloc[0]
v1_dt

Timestamp('2022-05-25 15:12:07.985000')

In [27]:
dt_history_pd_df[dt_history_pd_df["timestamp"] < v1_dt.strftime('%Y-%m-%d %H:%M:%S.%f')]["version"].iloc[0]

0

## 3.2. Viewing table history using Spark SQL

In [28]:
sparkSession.sql(f"DESCRIBE HISTORY delta.`{full_delta_table_path}`").toPandas()

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,1,2022-05-25 15:12:07.985,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,0.0,None,False,"{'numOutputRows': '5', 'numOutputBytes': '946'...",None
1,0,2022-05-25 15:10:48.657,None,None,WRITE,"{'mode': 'ErrorIfExists', 'partitionBy': '[]'}",None,None,None,NaN,None,True,"{'numOutputRows': '5', 'numOutputBytes': '735'...",None


## 3.3. Checkpoints

In short, checkpoints are a way to make delta lake more performant.

Once we’ve made a total of 10 commits to the transaction log, Delta Lake automatcally creates a checkpoint file andstores it in the \_delta_log subdirectory. This parquet file stores the entire state of the table at a given point in time. The idea is that this checkpoint provides Spark with a shortcut; rather than parsing all the json files and loading the the coresponding part files sequentiall, is simply loads the parquet file. If subsequent transactions do happen after the snapshot, Spark will filter out the irrelevant logs and only process the transactions occurring after the snapshot.

A more detailed article can be found [here](https://databricks.com/blog/2019/08/21/diving-into-delta-lake-unpacking-the-transaction-log.html).

# 4. Loading a specific versions of a delta table
In this section we will explore the ways in which we can load our data. We will see that it is possible to load a specific version of using a version number or a timestamp.

## 4.1. Load delta table using SparkSession.read() function

### 4.1.1. Load based on version number

We saw the \<version>.json file in the \_delta_log directory. This allows us to load a given version via it's index. Below we can see an example of loading the first version of data:

In [30]:
sparkSession.read.format("delta").option("versionAsOf", 0).load(full_delta_table_path).show()

+---+---+
|  A|  B|
+---+---+
|  1|  6|
|  2|  7|
|  3|  8|
|  4|  9|
|  5| 10|
+---+---+



In [31]:
sparkSession.read.format("delta").option("versionAsOf", 1).load(full_delta_table_path).show()

+---+---+---+
|  A|  B|  C|
+---+---+---+
|  1|  6| 11|
|  2|  7| 12|
|  3|  8| 13|
|  4|  9| 14|
|  5| 10| 15|
+---+---+---+



### 4.1.2. Load based on timestamp
Loading data based on a version number is not that user friendly or intuitive so instead we will look at loading data based on a timestamp.

#### 4.1.2.1. Note on possible errors

**Note**: There is a limited band of time for which we can pull data as of. There is a minimum and maximum datetime. If we try to pull a datetime that is outside this band, we will see an error thrown which looks like this:
```
Py4JJavaError: An error occurred while calling o1083.load.
: org.apache.spark.sql.delta.DeltaErrors$TimestampEarlierThanCommitRetentionException: The provided timestamp (2022-05-20 18:42:13.0) is before the earliest version available to this
table (2022-05-20 18:42:13.341). Please use a timestamp after 2022-05-20 18:42:13.
```


``` 
Py4JJavaError: An error occurred while calling o1131.load.
: org.apache.spark.sql.delta.DeltaErrors$TemporallyUnstableInputException: The provided timestamp: 2022-05-20 19:30:50.646 is after the latest commit timestamp of
2022-05-20 19:29:51.389. If you wish to query this version of the table, please either provide
the version with "VERSION AS OF 1" or use the exact timestamp
of the last commit: "TIMESTAMP AS OF '2022-05-20 19:29:51'".
```

**Note**: I have seen many articles claiming that we must specify the timestamp in yyyyMMddHHmmssSSS format. This is not true as we will see. The date must be a string in the format '%Y-%m-%d %H:%M:%S.%f'.

#### 4.1.2.1. Load data using timestamp

We first pull the timestamps from the history() function.

In [32]:
v0_timestamp = dt_history_pd_df[dt_history_pd_df["version"] == 0]["timestamp"].iloc[0]
v0_timestamp_str = v0_timestamp.strftime('%Y-%m-%d %H:%M:%S.%f')
print(v0_timestamp_str)

v1_timestamp = dt_history_pd_df[dt_history_pd_df["version"] == 1]["timestamp"].iloc[0]
v1_timestamp_str = v1_timestamp.strftime('%Y-%m-%d %H:%M:%S.%f')
print(v1_timestamp_str)

2022-05-25 15:10:48.657000
2022-05-25 15:12:07.985000


We load the timestamp associated with the first version and we see that the data is correct.

In [33]:
sparkSession.read \
  .format("delta") \
  .option("timestampAsOf", v0_timestamp_str) \
  .load(full_delta_table_path).show()

+---+---+
|  A|  B|
+---+---+
|  1|  6|
|  2|  7|
|  3|  8|
|  4|  9|
|  5| 10|
+---+---+



We then load data using a timestamp between the commit operations for v1 and v2. We see that the data resembles v1 as expected.

In [34]:
dt = (v1_timestamp - v0_timestamp)/2
mid_point_timestamp = v0_timestamp + dt
mid_point_timestamp_str = mid_point_timestamp.strftime('%Y-%m-%d %H:%M:%S.%f')

sparkSession.read \
  .format("delta") \
  .option("timestampAsOf", mid_point_timestamp_str) \
  .load(full_delta_table_path).show()

+---+---+
|  A|  B|
+---+---+
|  1|  6|
|  2|  7|
|  3|  8|
|  4|  9|
|  5| 10|
+---+---+



We can then see that we can load v2 data using the timestamp for v2.

In [35]:
sparkSession.read \
  .format("delta") \
  .option("timestampAsOf", v1_timestamp_str) \
  .load(full_delta_table_path).show()

+---+---+---+
|  A|  B|  C|
+---+---+---+
|  1|  6| 11|
|  2|  7| 12|
|  3|  8| 13|
|  4|  9| 14|
|  5| 10| 15|
+---+---+---+



## 4.2. Load using Spark SQL

We see that we can also load the delta tables using the Spark SQL API provided by the sql() function on the SparkSession object.

For this to work, we must provide the full path path of directory and not the relative path like we do with the read() function.

In [36]:
sparkSession.sql(f"SELECT * FROM delta.`{full_delta_table_path}`").toPandas()

,A,B,C
0,1,6,11
1,2,7,12
2,3,8,13
3,4,9,14
4,5,10,15


We can load specific versions by modifying the path to the table to contain a timestamp coresponding to the version of interest. The timestamp must be in the format yyyyMMddHHmmssSSS. As we will see, we are only able to see the first three digits of the microseconds field.

In [42]:
v0_timestamp_str_2 = v0_timestamp.strftime('%Y%m%d%H%M%S%f')
print(v0_timestamp_str_2)
p = f"{full_delta_table_path}@{v0_timestamp_str_2}"

sparkSession.sql(f"SELECT * FROM delta.`{p}`").toPandas()

20220525151048657000


AnalysisException: `/workspaces/crypto-data/jupyter-pod/ml-training-jupyter-notebooks/Example Data Sets/deltalake/time-travel-demo.delta@20220525151048657000` is not a Delta table.

Correcting the timestamp we see things work:

In [43]:
v0_timestamp_str_2 = v0_timestamp.strftime('%Y%m%d%H%M%S%f')[:-3]
print(v0_timestamp_str_2)
p = f"{full_delta_table_path}@{v0_timestamp_str_2}"

sparkSession.sql(f"SELECT * FROM delta.`{p}`").toPandas()

20220525151048657


,A,B
0,1,6
1,2,7
2,3,8
3,4,9
4,5,10


**Note**: Currently, loading a specific version using the SQL API using the keywords TIMESTAMP AS OF or VERSION AS OF is not supported. There is an open [github issue](https://github.com/delta-io/delta/issues/128) tracking progress.

# 5. Delta Table History Retention Policy

To time travel to a previous version, you must retain both the log and the data files for that version.

The data files backing a Delta table are never deleted automatically; data files are deleted only when you run VACUUM. VACUUM does not delete Delta log files; log files are automatically cleaned up after checkpoints are written.

So to summarize, deleteing data or logs is a manual process. If you run the vacuum() function you will only be able to time travel as far back as the retention policy.

The delta lake API provides several methods which impact the retention policy for a delta table. These settings appear to be set on the SparkSession level, not at the table level. 

According to the [documentation](https://docs.delta.io/latest/delta-utility.html#remove-files-no-longer-referenced-by-a-delta-table), the default retention period of log files is 30 days, configurable through the delta.logRetentionDuration property.

## 5.1. Viewing delta table properties
The retention policy for a table is stored as metadata in the table. At this point The only way to access the data is through the Spark SQL API.

### 5.1.2. Viewing delta table properties using Spark SQL

Viewing table properties is only possible throug the SparkSession.sql() function.

In [44]:
p = f"{full_delta_table_path}@{v0_timestamp_str_2}"

sparkSession.sql(f"SHOW TBLPROPERTIES delta.`{p}`").toPandas()

,key,value
0,delta.minReaderVersion,1
1,delta.minWriterVersion,2


If the table changes over time, we can see properties for specific versions by modifying the path as we saw before.

## 5.2. Setting or modifying delta table properties
When we modify table properties we are committing a new version of the table.

### 5.2.2. Setting table properties using SQL API

We can run the following query to arbitratily set various table properties

In [45]:
log_retention_duration_days = "interval 36500000 days"
delete_file_retention_duration = "interval 36500000 days"

sparkSession.sql(f"""
ALTER TABLE delta.`{full_delta_table_path}` SET TBLPROPERTIES (
  delta.logRetentionDuration='{log_retention_duration_days}', 
  delta.deletedFileRetentionDuration='{delete_file_retention_duration}'
)
""")
    
sparkSession.sql(f"SHOW TBLPROPERTIES delta.`{full_delta_table_path}`").toPandas()

,key,value
0,delta.logRetentionDuration,interval 36500000 days
1,delta.minReaderVersion,1
2,delta.minWriterVersion,2
3,delta.deletedFileRetentionDuration,interval 36500000 days


We see that altering the table results in a new transaction being written to the transaction log. Loading the history shows us this the details of this new delta.

In [46]:
sparkSession.sql(f"DESCRIBE HISTORY delta.`{full_delta_table_path}`").toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,2,2022-05-25 15:20:09.222,None,None,SET TBLPROPERTIES,"{'properties': '{""delta.logRetentionDuration"":...",None,None,None,1.0,None,True,{},None
1,1,2022-05-25 15:12:07.985,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,0.0,None,False,"{'numOutputRows': '5', 'numOutputBytes': '946'...",None
2,0,2022-05-25 15:10:48.657,None,None,WRITE,"{'mode': 'ErrorIfExists', 'partitionBy': '[]'}",None,None,None,NaN,None,True,"{'numOutputRows': '5', 'numOutputBytes': '735'...",None


We can also see that the table properties are only effective as of a specific version. Looking at the properties of the old version shows that these properties are not present.

In [47]:
v1_timestamp_str_2 = v1_timestamp.strftime('%Y%m%d%H%M%S%f')[:-3]
p = f"{full_delta_table_path}@{v1_timestamp_str_2}"

sparkSession.sql(f"SHOW TBLPROPERTIES delta.`{p}`").toPandas()

,key,value
0,delta.minReaderVersion,1
1,delta.minWriterVersion,2


### 5.2.3. Setting table propeties using SparkConf

We can also set table properties globally using the SparkConf object. In doing so, any new tables that are created will inherit these properties. This may be useful when trying to enforce certain behaviors. But note, this will not change the properties of existing tables.

Below we can see an example code snippet:

```python
sparkSession.conf.set(
  "spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite", "true")
``` 

## 5.3. Deleting Delta Table History


As mentioned previously, the delta table is stored in parts and assembled from those parts. At some point we may want to do a cleanup and delete old files so that the footprint of the table shrinks.

You can remove files no longer referenced by a Delta table and are older than the retention threshold by running the vacuum() command on the table.

Vacuum deletes only data files, not log files. Log files are deleted automatically and asynchronously after checkpoint operations.

We will look more at the retention policy in a later section.

# 6. Restoration Operations
We saw previously that we can load a specific version of data by specifying a version number or associated timestamp. But in some cases, we may want to manipulate our data and revert it to a prior state. An example might be that a problematic ETL process erroneously correpted some data or returned thewrong schema/values. With delta lake, this is an easy command. Resore operations behave like other transactions; the restore creates a new version which looks identical to the desired version.

**Note**: The documentation shows several methods for restoring a table using either the DeltaLake object or the Spark SQL API. These methods unfortunately do not work with the current version of delta lake (1.0.1). As a workaround we can simply load a previous version and then write that to the delta lake.

More informationregarding restors can be found [here](
https://docs.delta.io/latest/delta-utility.html#restore-a-delta-table-to-an-earlier-state).

In [48]:
old_table = sparkSession.read.format("delta").option("versionAsOf", 0).load(full_delta_table_path)
old_table.write\
  .format("delta")\
  .mode("overwrite")\
  .option("overwriteSchema", "true")\
  .save(full_delta_table_path)

sparkSession.read.format("delta").load(full_delta_table_path).show()

+---+---+
|  A|  B|
+---+---+
|  1|  6|
|  2|  7|
|  3|  8|
|  4|  9|
|  5| 10|
+---+---+



We can see a new version was written.

In [49]:
sparkSession.sql(f"DESCRIBE HISTORY delta.`{full_delta_table_path}`").toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,3,2022-05-25 15:20:54.991,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,2.0,None,False,"{'numOutputRows': '5', 'numOutputBytes': '735'...",None
1,2,2022-05-25 15:20:09.222,None,None,SET TBLPROPERTIES,"{'properties': '{""delta.logRetentionDuration"":...",None,None,None,1.0,None,True,{},None
2,1,2022-05-25 15:12:07.985,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,0.0,None,False,"{'numOutputRows': '5', 'numOutputBytes': '946'...",None
3,0,2022-05-25 15:10:48.657,None,None,WRITE,"{'mode': 'ErrorIfExists', 'partitionBy': '[]'}",None,None,None,NaN,None,True,"{'numOutputRows': '5', 'numOutputBytes': '735'...",None
